In [1]:
## Heatmap
### Number of sales by Zip, layering in demographic information

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import folium
import json
import csv
import os

# Import API key
from api_keys import g_key

In [3]:
# Source data location
housingFile = 'source_data/housing_data_cleaned.csv'

# Read in source
housingDF = pd.read_csv(housingFile, low_memory=False)
housingDF.head()
#housingDF.dtypes

,basements,building_code_description,category_code_description,census_tract,central_air,depth,exempt_building,exempt_land,exterior_condition,fireplaces,...,topography,total_area,total_livable_area,type_heater,unit,view_type,year_built,year_built_estimate,zip_code,zoning
0,D,ROW 3 STY MASONRY,Single Family,241.0,N,67.0,49200,0,4.0,0.0,...,F,938.00,1344.0,A,NaN,I,1895,Y,19144.0,RSA5
1,NaN,RES CONDO 3 STY MAS+OTH,Single Family,337.0,Y,0.0,45000,0,4.0,0.0,...,NaN,0.00,947.0,NaN,B307,I,1970,Y,19152.0,RM2
2,NaN,ROW 2 STY MASONRY,Single Family,201.0,NaN,70.0,0,0,4.0,0.0,...,F,1044.00,1190.0,NaN,NaN,I,1940,Y,19140.0,RM1
3,H,ROW B/GAR 2 STY MASONRY,Single Family,281.0,N,95.5,0,0,4.0,0.0,...,F,1686.53,1633.0,B,NaN,I,1940,Y,19141.0,RSA3
4,NaN,ROW 2 STY MASONRY,Single Family,293.0,NaN,112.5,0,0,4.0,0.0,...,F,2165.62,1320.0,B,NaN,I,1940,Y,19124.0,RSA5


In [4]:
# Summary table for housing dataframe
# include location, market_value, sale_date, sale_price, year_built, zip_code

housingDFnew = housingDF[['location', 'market_value', 'sale_date', 'sale_price', 'year_built', 'zip_code', 'zoning']].copy()
housingDFnew.head()

,location,market_value,sale_date,sale_price,year_built,zip_code,zoning
0,5832 KNOX ST,150600,44102,90000,1895,19144.0,RSA5
1,2201 STRAHLE ST,85000,44102,145000,1970,19152.0,RM2
2,3810 N SYDENHAM ST,48000,44099,52500,1940,19140.0,RM1
3,5310 N BROAD ST,104000,44099,154000,1940,19141.0,RSA3
4,4286 GRISCOM ST,65200,44098,65900,1940,19124.0,RSA5


In [5]:
# Formatting Date
housingDFnew['sale_date'].head()

0    44102
1    44102
2    44099
3    44099
4    44098
Name: sale_date, dtype: int64

In [6]:
# Converting
housingDFnew['sale_date'] = pd.to_datetime(housingDFnew['sale_date'], unit='D', origin='1899-12-30')
housingDFnew['sale_date'].head()

0   2020-09-28
1   2020-09-28
2   2020-09-25
3   2020-09-25
4   2020-09-24
Name: sale_date, dtype: datetime64[ns]

In [7]:
# Bring in demographics dataframe
demographicsFile = 'source_data/philly_demographics.csv'

# Read in demographics df
demographicsDF = pd.read_csv(demographicsFile)
demographicsDF.head()

,City,Zip,NABE,Lat,Long,MEDINC,P_WHITE,P_BLACK,P_HISP,P_ASIAN,P_OTHERS,P_MinCOMBINED
0,Philadelphia,19102,Center City,39.948498,-75.16683,90750,0.73,0.04,0.05,0.16,0.01,0.27
1,Philadelphia,19103,Center City West,39.953663,-75.17399,73611,0.72,0.06,0.06,0.12,0.04,0.28
2,Philadelphia,19103,Center City West,39.953860,-75.16713,73611,0.72,0.06,0.06,0.12,0.04,0.28
3,Philadelphia,19104,"University City, Mantua, Powelton",39.956417,-75.20855,25865,0.35,0.42,0.05,0.14,0.04,0.65
4,Philadelphia,19106,"Old City, Society Hill",39.949120,-75.14397,109393,0.77,0.08,0.06,0.06,0.02,0.23


In [8]:
# Make the name of the zip column the same
demographicsDF_update = demographicsDF.rename(columns={'Zip': 'zip_code'})
demographicsDF_update.head()

,City,zip_code,NABE,Lat,Long,MEDINC,P_WHITE,P_BLACK,P_HISP,P_ASIAN,P_OTHERS,P_MinCOMBINED
0,Philadelphia,19102,Center City,39.948498,-75.16683,90750,0.73,0.04,0.05,0.16,0.01,0.27
1,Philadelphia,19103,Center City West,39.953663,-75.17399,73611,0.72,0.06,0.06,0.12,0.04,0.28
2,Philadelphia,19103,Center City West,39.953860,-75.16713,73611,0.72,0.06,0.06,0.12,0.04,0.28
3,Philadelphia,19104,"University City, Mantua, Powelton",39.956417,-75.20855,25865,0.35,0.42,0.05,0.14,0.04,0.65
4,Philadelphia,19106,"Old City, Society Hill",39.949120,-75.14397,109393,0.77,0.08,0.06,0.06,0.02,0.23


In [9]:
# Merge dataframes on column (zip)
mergedDF = pd.merge(housingDFnew, demographicsDF_update, on='zip_code', how='inner')
mergedDF

,location,market_value,sale_date,sale_price,year_built,zip_code,zoning,City,NABE,Lat,Long,MEDINC,P_WHITE,P_BLACK,P_HISP,P_ASIAN,P_OTHERS,P_MinCOMBINED
0,5832 KNOX ST,150600,2020-09-28,90000,1895,19144.0,RSA5,Philadelphia,Germantown,40.033259,-75.177865,33786,0.16,0.75,0.04,0.02,0.03,0.84
1,5832 KNOX ST,150600,2020-09-28,90000,1895,19144.0,RSA5,Philadelphia,Germantown,40.030370,-75.165700,33786,0.16,0.75,0.04,0.02,0.03,0.84
2,528 BRINTON ST,23000,2020-09-19,63000,1925,19144.0,RSA3,Philadelphia,Germantown,40.033259,-75.177865,33786,0.16,0.75,0.04,0.02,0.03,0.84
3,528 BRINTON ST,23000,2020-09-19,63000,1925,19144.0,RSA3,Philadelphia,Germantown,40.030370,-75.165700,33786,0.16,0.75,0.04,0.02,0.03,0.84
4,148 HANSBERRY ST,106400,2020-09-18,95000,1920,19144.0,RSA3,Philadelphia,Germantown,40.033259,-75.177865,33786,0.16,0.75,0.04,0.02,0.03,0.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130845,1521 SPRUCE ST,1383800,2016-02-01,2318000,1890,19102.0,CMX4,Philadelphia,Center City,39.948498,-75.166830,90750,0.73,0.04,0.05,0.16,0.01,0.27
130846,50 S 16TH ST,1170600,2016-01-14,1239000,1990,19102.0,NaN,Philadelphia,Center City,39.948498,-75.166830,90750,0.73,0.04,0.05,0.16,0.01,0.27
130847,50 S 16TH ST,800400,2016-01-14,846000,1990,19102.0,NaN,Philadelphia,Center City,39.948498,-75.166830,90750,0.73,0.04,0.05,0.16,0.01,0.27
130848,1414 S PENN SQ,2063300,2016-01-14,2150000,2009,19102.0,CMX5,Philadelphia,Center City,39.948498,-75.166830,90750,0.73,0.04,0.05,0.16,0.01,0.27


In [10]:
avgSaleDF = mergedDF[['location', 'market_value', 'sale_price', 'MEDINC', 'zip_code', 'Lat', 'Long']].copy()
avgSaleDF.head()

,location,market_value,sale_price,MEDINC,zip_code,Lat,Long
0,5832 KNOX ST,150600,90000,33786,19144.0,40.033259,-75.177865
1,5832 KNOX ST,150600,90000,33786,19144.0,40.030370,-75.165700
2,528 BRINTON ST,23000,63000,33786,19144.0,40.033259,-75.177865
3,528 BRINTON ST,23000,63000,33786,19144.0,40.030370,-75.165700
4,148 HANSBERRY ST,106400,95000,33786,19144.0,40.033259,-75.177865


In [11]:
# Sort avgSaleDF by sale price
avgSaleDF.sort_values(by=['sale_price', 'zip_code'])

# Reindex dataframe
avgSaleDF.reset_index(drop=True)

,location,market_value,sale_price,MEDINC,zip_code,Lat,Long
0,5832 KNOX ST,150600,90000,33786,19144.0,40.033259,-75.177865
1,5832 KNOX ST,150600,90000,33786,19144.0,40.030370,-75.165700
2,528 BRINTON ST,23000,63000,33786,19144.0,40.033259,-75.177865
3,528 BRINTON ST,23000,63000,33786,19144.0,40.030370,-75.165700
4,148 HANSBERRY ST,106400,95000,33786,19144.0,40.033259,-75.177865
...,...,...,...,...,...,...,...
130845,1521 SPRUCE ST,1383800,2318000,90750,19102.0,39.948498,-75.166830
130846,50 S 16TH ST,1170600,1239000,90750,19102.0,39.948498,-75.166830
130847,50 S 16TH ST,800400,846000,90750,19102.0,39.948498,-75.166830
130848,1414 S PENN SQ,2063300,2150000,90750,19102.0,39.948498,-75.166830


In [12]:
print(avgSaleDF.dtypes)
#print(mergedDF.sale_price)
#print(mergedDF.MEDINC)

location         object
market_value      int64
sale_price        int64
MEDINC            int64
zip_code        float64
Lat             float64
Long            float64
dtype: object


In [44]:
# Aggregate total sales by zip code

# Create df to use for grouping
avgSaleDFnew = mergedDF[['zip_code','sale_price']]

avgSaleDFnew = avgSaleDFnew.dropna(how='any')

# Convert to string in main ETL
avgSaleDFnew['zip_code'] = avgSaleDFnew['zip_code'].astype('Int64').astype('str')


# Create groupby object
avgSaleDFnewGroup = avgSaleDFnewGroup.groupby('zip_code')

# Create grouped data frame with count of sales, mean, median sale price
avgSaleDFnewGroup = avgSaleDFnewGroup.count()
avgSaleDFnewGroup = avgSaleDFnewGroup.reset_index()
avgSaleDFnewGroup

# Add zip codes 19109 and 19102, so that GeoJSON will work - not in original df but in GeoJSON file
avgSaleDFnewGroup = avgSaleDFnewGroup.append({'zip_code':'19109','sale_price':0},ignore_index=True)
avgSaleDFnewGroup = avgSaleDFnewGroup.append({'zip_code':'19112','sale_price':0},ignore_index=True)

#summary_zip_groupby_df.head()
avgSaleDFnewGroup['zip_code'].value_counts()

19109    2
19112    2
19140    1
19148    1
19106    1
19119    1
19121    1
19129    1
19123    1
19143    1
19114    1
19118    1
19142    1
19127    1
19145    1
19151    1
19144    1
19128    1
19137    1
19104    1
19102    1
19149    1
19103    1
19152    1
19138    1
19146    1
19147    1
19154    1
19111    1
19107    1
19124    1
19133    1
19131    1
19120    1
19153    1
19130    1
19136    1
19116    1
Name: zip_code, dtype: int64

In [14]:
## Mapping

In [34]:
# file name
zips_geo = r'source_data/Zipcodes_Poly.geojson'

# creating base of map using geolocations
avgSaleMap = folium.Map(location=[39.9526, -75.1652], zoom_start = 11)

# Creating map
avgSaleMap.choropleth(
    geo_data=zips_geo,
    data=avgSaleDFnewGroup,
    columns=['zip_code', 'sale_price'],
    legend_name='Average Sale Price by Zip Code',
    key_on='feature.properties.CODE',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,)

# Display
folium.LayerControl().add_to(avgSaleMap)


avgSaleMap

JSONDecodeError: Expecting value: line 7 column 1 (char 6)

In [ ]:
# Adding markers to the map
info_box = """
<d1>
<dt>Neighborhood:</dt><dd>{NABE}</dd>
<dt>Zip code:</dt><dd>{zip_code}</dd>
<dt>Sale price:</dt><dd>{sale_price}</dd>
<dt>Median income:</dt><dd>{MEDINC}</dd>
"""

## Storing the df row
# house_info = [info_box.format(**row) for index, row in heatmapDF.iterrows()]

# locations = heatmapDF[["Lat", "Long"]]

In [ ]:
## Add marker layer ontop of heat map
#markers = gmaps.marker_layer(locations)
#fig.add_layer(markers)

## Display figure
#fig